In [1]:
from typing import List
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, Aer
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate

In [ ]:
backend = Aer.get_backend('aer_simulator')

In [ ]:
def oracle1() -> QuantumCircuit:
    """Oracle 1 for assignment 3 Q1 (a)

    Returns:
        Oracle circuit.
    """
    data_reg = QuantumRegister(3)
    ancilla = QuantumRegister(1)
    circuit = QuantumCircuit(data_reg, ancilla, name='Oracle1')
    
    circuit.x(data_reg[1])
    circuit.mcx(data_reg, ancilla)
    circuit.x(data_reg[1])
    return circuit


def oracle2() -> QuantumCircuit:
    """Oracle 2 for assignment 3 Q1 (a)

    Returns:
        Oracle circuit.
    """
    data_reg = QuantumRegister(3)
    ancilla = QuantumRegister(1)
    circuit = QuantumCircuit(data_reg, ancilla, name='Oracle2')
    
    circuit.x(data_reg[1])
    circuit.mcx(data_reg, ancilla)
    circuit.x(data_reg[1])
    circuit.x(data_reg[0])
    circuit.mcx(data_reg, ancilla)
    circuit.x(data_reg[0])
    return circuit


def search(num_data):
    data_reg = QuantumRegister(num_data)
    circuit = QuantumCircuit(data_reg)
    circuit.h(data_reg)
    circuit.x(data_reg)
    circuit.h(data_reg[-1])
    circuit.mcx(data_reg[:-1], data_reg[-1])
    circuit.h(data_reg[-1])
    circuit.x(data_reg)
    circuit.h(data_reg)
    return circuit


def build_circuit():
    num_data_bits = 3
    data_reg = QuantumRegister(num_data_bits)
    ancilla = QuantumRegister(1)
    c_reg = ClassicalRegister(len(data_reg))
    circuit = QuantumCircuit(data_reg, ancilla, c_reg)
    circuit.h(data_reg)
    circuit.x(ancilla)
    circuit.h(ancilla)
    oracle_circuit = oracle1()
    search_circuit = search(len(data_reg))
    circuit.append(oracle_circuit, data_reg[:] + ancilla[:])
    circuit.append(search_circuit, data_reg)
    circuit.append(oracle_circuit, data_reg[:] + ancilla[:])
    circuit.append(search_circuit, data_reg)
    print(search_circuit)
    circuit.measure(data_reg, c_reg)
    return circuit
    
circuit = build_circuit()
print(circuit)
# Statevector(circuit).draw(output='latex')
counts = backend.run(transpile(circuit, backend), shots=1000).result().get_counts()
print(counts)

      ┌───┐┌───┐          ┌───┐┌───┐     
q4_0: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
      ├───┤├───┤       │  ├───┤├───┤     
q4_1: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
      ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐
q4_2: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
      └───┘└───┘└───┘└───┘└───┘└───┘└───┘
      ┌───┐     ┌──────────┐┌─────────────┐┌──────────┐┌─────────────┐┌─┐      
q0_0: ┤ H ├─────┤0         ├┤0            ├┤0         ├┤0            ├┤M├──────
      ├───┤     │          ││             ││          ││             │└╥┘┌─┐   
q0_1: ┤ H ├─────┤1         ├┤1 circuit-82 ├┤1         ├┤1 circuit-82 ├─╫─┤M├───
      ├───┤     │  Oracle1 ││             ││  Oracle1 ││             │ ║ └╥┘┌─┐
q0_2: ┤ H ├─────┤2         ├┤2            ├┤2         ├┤2            ├─╫──╫─┤M├
      ├───┤┌───┐│          │└─────────────┘│          │└─────────────┘ ║  ║ └╥┘
  q1: ┤ X ├┤ H ├┤3         ├───────────────┤3         ├────────────────╫──╫──╫─
      └───┘└───┘└──────────┘               └──────────┘           

In [9]:
def qram(data: List) -> QuantumCircuit:
    """A naive implementation of QRAM for assigment 3 Q1 (c).
    Given the data list, it prepares the state \sum_x |x>|D(x)>,
    where the |x> is the address (index) and |D(x)> is the data in the corresponding address,
    e,g, if the list is [3, 5, 1], then the state is (|0>|3> + |1>|5> + |2>|1>) / sqrt(3).

    Args:
        data: The Python List object of the data you want to search.

    Returns:
        circuit: QuantumCircuit for QRAM.
    """
    num_addr_bits = int(np.ceil(np.log2(len(data))))
    num_data_bits = int(np.ceil(np.log2(max(data))))
    address_reg = QuantumRegister(num_addr_bits)
    data_reg = QuantumRegister(num_data_bits)
    circuit = QuantumCircuit(address_reg, data_reg, name='QRAM')
    
    for id_addr in range(len(data)):
        # load address index
        id_str = format(id_addr, 'b').zfill(len(address_reg))
        for i_id_str, b_id_str in enumerate(id_str):
            if b_id_str == '0': circuit.x(address_reg[i_id_str])
        # load data
        data_str = format(data[id_addr], 'b').zfill(len(data_reg))
        for i_data_str, b_data_str in enumerate(data_str):
            if b_data_str == '1': circuit.mct(address_reg, data_reg[i_data_str])

        for i_id_str, b_id_str in enumerate(id_str):
            if b_id_str == '0': circuit.x(address_reg[i_id_str])
        circuit.barrier()
    return circuit


def oracle_c(num_data: int) -> QuantumCircuit:
    """Oracle for assignment 3 Q1 (c).
    This oracle is used for finding if the data == 5, not the index.

    Args:
        num_data: Number of qubits in data register.

    Returns:
        Oracle circuit.
    """
    data_reg = QuantumRegister(num_data)
    ancilla = QuantumRegister(1)
    circuit = QuantumCircuit(data_reg, ancilla, name='Oracle')
    
    circuit.x(data_reg[1])
    circuit.mcx(data_reg, ancilla)
    circuit.x(data_reg[1])
    return circuit


def search(num_data):
    data_reg = QuantumRegister(num_data)
    circuit = QuantumCircuit(data_reg)
    circuit.h(data_reg)
    circuit.x(data_reg)
    circuit.h(data_reg[-1])
    circuit.mcx(data_reg[:-1], data_reg[-1])
    circuit.h(data_reg[-1])
    circuit.x(data_reg)
    circuit.h(data_reg)
    return circuit


def build_circuit():
    data = [5, 2, 4, 5, 3, 7, 6, 1]
    num_data_bits = int(np.ceil(np.log2(max(data))))
    address_reg = QuantumRegister(int(np.ceil(np.log2(len(data)))))
    data_reg = QuantumRegister(num_data_bits)
    ancilla = QuantumRegister(1)
    c_reg = ClassicalRegister(len(address_reg))
    circuit = QuantumCircuit(address_reg, data_reg, ancilla, c_reg)
    circuit.h(address_reg)
    circuit.x(ancilla)
    circuit.h(ancilla)
    qram_circuit = qram(data)
    oracle_circuit = oracle_c(num_data_bits)
    search_circuit = search(len(address_reg))
    circuit.append(qram_circuit, address_reg[:] + data_reg[:])
    circuit.append(oracle_circuit, data_reg[:] + ancilla[:])
    circuit.append(qram_circuit, address_reg[:] + data_reg[:])
    circuit.append(search_circuit, address_reg)
    
    # circuit.append(qram_circuit, address_reg[:] + data_reg[:])
    # circuit.append(oracle_circuit, data_reg[:] + ancilla[:])
    # circuit.append(qram_circuit, address_reg[:] + data_reg[:])
    # circuit.append(search_circuit, address_reg)
    
    circuit.measure(address_reg, c_reg)
    return circuit
    
circuit = build_circuit()
print(circuit)
# Statevector(circuit.reverse_bits()).draw(output='latex')
counts = backend.run(transpile(circuit, backend), shots=10000).result().get_counts()
print(counts)

       ┌───┐┌───────┐           ┌───────┐┌──────────────┐┌─┐      
q30_0: ┤ H ├┤0      ├───────────┤0      ├┤0             ├┤M├──────
       ├───┤│       │           │       ││              │└╥┘┌─┐   
q30_1: ┤ H ├┤1      ├───────────┤1      ├┤1 circuit-404 ├─╫─┤M├───
       ├───┤│       │           │       ││              │ ║ └╥┘┌─┐
q30_2: ┤ H ├┤2      ├───────────┤2      ├┤2             ├─╫──╫─┤M├
       └───┘│  QRAM │┌─────────┐│  QRAM │└──────────────┘ ║  ║ └╥┘
q31_0: ─────┤3      ├┤0        ├┤3      ├─────────────────╫──╫──╫─
            │       ││         ││       │                 ║  ║  ║ 
q31_1: ─────┤4      ├┤1        ├┤4      ├─────────────────╫──╫──╫─
            │       ││  Oracle ││       │                 ║  ║  ║ 
q31_2: ─────┤5      ├┤2        ├┤5      ├─────────────────╫──╫──╫─
       ┌───┐└─┬───┬─┘│         │└───────┘                 ║  ║  ║ 
  q32: ┤ X ├──┤ H ├──┤3        ├──────────────────────────╫──╫──╫─
       └───┘  └───┘  └─────────┘                          ║  ║

---

In [10]:
def build_circuit():
    data = [5, 2, 4, 5, 3, 7, 6, 1]
    num_data_bits = int(np.ceil(np.log2(max(data))))
    address_reg = QuantumRegister(int(np.ceil(np.log2(len(data)))))
    data_reg = QuantumRegister(num_data_bits)
    circuit = QuantumCircuit(address_reg, data_reg)
    circuit.h(address_reg)
    qram_circuit = qram(data)
    circuit.append(qram_circuit, address_reg[:] + data_reg[:])
    
    return circuit
    
circuit = build_circuit()
print(circuit)
Statevector(circuit.reverse_bits()).draw(output='latex')

       ┌───┐┌───────┐
q58_0: ┤ H ├┤0      ├
       ├───┤│       │
q58_1: ┤ H ├┤1      ├
       ├───┤│       │
q58_2: ┤ H ├┤2      ├
       └───┘│  QRAM │
q59_0: ─────┤3      ├
            │       │
q59_1: ─────┤4      ├
            │       │
q59_2: ─────┤5      ├
            └───────┘


<IPython.core.display.Latex object>